<a href="https://colab.research.google.com/github/RaisaAnika/model-confidence_CIFAR10/blob/main/CIFAR10_modelconfidence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



**Library Installations**


In [31]:
!sudo pip install Pillow
!sudo pip install imagehash
!pip install tensorflow-privacy


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [32]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import glob
import itertools
import collections

from PIL import Image
import cv2
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import numpy as np
import torch
import imagehash
import hashlib


#importing libraries
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
import sys
from matplotlib import pyplot
from keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from tensorflow.keras.optimizers import SGD
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split 

# general imports
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from scipy import special

# tensorflow imports
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, MaxPooling2D,Dropout, BatchNormalization, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
# tensorflow-privacy libraries
from tensorflow_privacy.privacy.membership_inference_attack import membership_inference_attack as mia
from tensorflow_privacy.privacy.membership_inference_attack.data_structures import AttackInputData
from tensorflow_privacy.privacy.membership_inference_attack.data_structures import SlicingSpec
from tensorflow_privacy.privacy.membership_inference_attack.data_structures import AttackType

**DataSet Selection**

In [34]:
dataset='Cifar10'

In [35]:
def load_cifar10():
  """Loads cifar10-Dataset and preprocesses to combine training and test data."""
  
  # load the existing CIFAR10 dataset that comes in form of traing + test data and labels
  train, test = tf.keras.datasets.cifar10.load_data()
  train_data, train_labels = train
  test_data, test_labels = test

  # scale the images from color values 0-255 to numbers from 0-1 to help the training process
  train_data1 = np.array(train_data, dtype=np.float32) / 255
  test_data1 = np.array(test_data, dtype=np.float32) / 255

  # cifar10 labels come one-hot encoded, there
  train_labels = train_labels.flatten()
  test_labels = test_labels.flatten()

  return train_data1, train_labels, test_data1, test_labels, train_data, train_labels, test_data, test_labels 

In [36]:
train_data, train_labels, test_data, test_labels,trainX1,trainY1, testX1,testY1 = load_cifar10()  

In [37]:
train_data

array([[[[0.23137255, 0.24313726, 0.24705882],
         [0.16862746, 0.18039216, 0.1764706 ],
         [0.19607843, 0.1882353 , 0.16862746],
         ...,
         [0.61960787, 0.5176471 , 0.42352942],
         [0.59607846, 0.49019608, 0.4       ],
         [0.5803922 , 0.4862745 , 0.40392157]],

        [[0.0627451 , 0.07843138, 0.07843138],
         [0.        , 0.        , 0.        ],
         [0.07058824, 0.03137255, 0.        ],
         ...,
         [0.48235294, 0.34509805, 0.21568628],
         [0.46666667, 0.3254902 , 0.19607843],
         [0.47843137, 0.34117648, 0.22352941]],

        [[0.09803922, 0.09411765, 0.08235294],
         [0.0627451 , 0.02745098, 0.        ],
         [0.19215687, 0.10588235, 0.03137255],
         ...,
         [0.4627451 , 0.32941177, 0.19607843],
         [0.47058824, 0.32941177, 0.19607843],
         [0.42745098, 0.28627452, 0.16470589]],

        ...,

        [[0.8156863 , 0.6666667 , 0.3764706 ],
         [0.7882353 , 0.6       , 0.13333334]

In [38]:
train_y=to_categorical(train_labels)
test_y=to_categorical(test_labels)

MODEL APPLICATION

In [47]:
shape = (32, 32, 3)
num_class=10
learning_rate=.01
epochs=2 #60
batch_size=128
M=5

In [40]:
def AlexnetModel(input_shape,num_classes):
    model = Sequential()
# model.add(Conv2D(96, (11,11), strides=(4,4), activation='relu', padding='same', input_shape=(img_height, img_width, channel,)))
# for original Alexnet
    model.add(Conv2D(48, (3,3), strides=(2,2), activation='relu', padding='same', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
# Local Response normalization for Original Alexnet
    model.add(BatchNormalization())

    model.add(Conv2D(96, (3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2)))
# Local Response normalization for Original Alexnet
    model.add(BatchNormalization())

    model.add(Conv2D(192, (3,3), activation='relu', padding='same'))
    model.add(Conv2D(192, (3,3), activation='relu', padding='same'))
    model.add(Conv2D(256, (3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2)))
# Local Response normalization for Original Alexnet
    model.add(BatchNormalization())

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    return model

In [41]:
model_np = AlexnetModel(shape,num_class)
model_np.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 16, 16, 48)        1344      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 8, 8, 48)         0         
 2D)                                                             
                                                                 
 batch_normalization_6 (Batc  (None, 8, 8, 48)         192       
 hNormalization)                                                 
                                                                 
 conv2d_11 (Conv2D)          (None, 8, 8, 96)          41568     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 3, 3, 96)         0         
 2D)                                                             
                                                      

In [42]:
#Run the model
# specify parameters
#learning_rate=.005
optimizer = tf.keras.optimizers.SGD(learning_rate)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

# compile the model
model_np.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [43]:
# train the model
history = model_np.fit(train_data, train_y,
                       validation_data=(test_data, test_y),
                       batch_size=128, 
                       epochs=epochs)

Epoch 1/2
391/391 [==============================] - 215s 539ms/step - loss: 1.9475 - accuracy: 0.3078 - val_loss: 1.9700 - val_accuracy: 0.2692
Epoch 2/2
391/391 [==============================] - 198s 506ms/step - loss: 1.5247 - accuracy: 0.4488 - val_loss: 1.3514 - val_accuracy: 0.5105


In [44]:
# since we have not specified an activation function on the last layer
# calling the predict function returns the logits
print('Predict on train...')
prob_train = model_np.predict(train_data[0:5000])
print('Predict on test...')
prob_test = model_np.predict(test_data[5000:10000])

Predict on train...
157/157 [==============================] - 5s 29ms/step
Predict on test...
157/157 [==============================] - 3s 22ms/step


In [45]:
train=train_data[0:5000]
trainX=trainX1[0:5000]
test=test_data[5000:10000]
testX=testX1[5000:10000]

Training MIAShield Models 

In [48]:
from sklearn.model_selection import StratifiedKFold
X=train_data
Y=train_labels
for i in range(0,M):
  K=M-i
  if(K>=2):
    train=X
    test=Y
    skf = StratifiedKFold(n_splits=K)
    skf.get_n_splits(X, Y)
    #print(skf)
    for train_index, test_index in skf.split(X, Y):
       #print(X.shape)
       X,globals()['x_train_split%s' % i]=train[train_index], train[test_index]
       Y,globals()['trainy_split%s' % i]=test[train_index], test[test_index]
       y_hot=globals()['trainy_split%s' % i]
       y_hot=to_categorical(y_hot)
       globals()['y_train_split%s' % i]=y_hot
       #X=train
       #Y=test
  elif K==1:
    globals()['x_train_split%s' % i]=X
    globals()['trainy_split%s' % i]=Y
    y_hot=to_categorical(Y)
    globals()['y_train_split%s' % i]=y_hot

In [50]:
#create datagenerator for augmented training
from keras.preprocessing.image import ImageDataGenerator
width_shift = 3/32
height_shift = 3/32
flip = True
rotation=10
datagen = ImageDataGenerator(
    horizontal_flip=flip,
    width_shift_range=.1,
    height_shift_range=.1,
    #brightness_range=[0.02,.1],
    rotation_range=rotation,
    #zoom_range=[0.5,1.0]
    shear_range=0.2,
    zoom_range=0.2,
    )

In [52]:
for iter in range(0,M):
  modelt=AlexnetModel(shape,num_class)
  optimizer = tf.keras.optimizers.SGD(learning_rate=.01)
  loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
# compile the model
  modelt.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
  #create augmented data
  data_t=globals()['x_train_split%s' % iter]
  target_t=globals()['y_train_split%s' % iter]
  dataGen=datagen
  dataGen.fit(data_t)
  #train=datagen.flow(data_t, target_t, batch_size=128)


In [54]:
# Fit the model
globals()['history_%s' % iter] = modelt.fit_generator(dataGen.flow(data_t, target_t, batch_size=128, shuffle=False),
                    steps_per_epoch = 10000 / 128, epochs=4, validation_data=(test_data, test_y))
globals()['model_%s' % iter]=modelt

# # Fit the model
#   globals()['history_%s' % iter] = modelt.fit_generator(dataGen.flow(data_t, target_t, batch_size=128, shuffle=False),
#                     steps_per_epoch = 10000 / 128, epochs=epochs+30, validation_data=(test_data, test_y))
#   globals()['model_%s' % iter]=modelt

Epoch 1/4


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  This is separate from the ipykernel package so we can avoid doing imports until


78/78 [==============================] - 52s 657ms/step - loss: 2.3890 - accuracy: 0.1873 - val_loss: 2.2930 - val_accuracy: 0.1179
Epoch 2/4
78/78 [==============================] - 47s 607ms/step - loss: 2.0999 - accuracy: 0.2520 - val_loss: 2.2985 - val_accuracy: 0.1159
Epoch 3/4
78/78 [==============================] - 47s 607ms/step - loss: 1.9532 - accuracy: 0.2854 - val_loss: 2.3075 - val_accuracy: 0.1175
Epoch 4/4
78/78 [==============================] - 50s 642ms/step - loss: 1.8662 - accuracy: 0.3182 - val_loss: 2.2071 - val_accuracy: 0.1926


In [55]:
#calculate accuracy/utility test acc
prb=model_0.predict(test_data[0:5000])
test_labeld_result=np.argmax(prb,axis=1)
accuracy_score(test_labels[0:5000],test_labeld_result)
#prob_testd

NameError: ignored